In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
import sys
import re
from citymobil_python_mysql_wrapper import MysqlWrapper
import pyexasol
from random import randrange
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.power import tt_ind_solve_power, GofChisquarePower
from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.stats import norm, chi2_contingency

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [3]:
def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.mean, # интересующая нас статистика
    bootstrap_conf_level = 0.99 # уровень значимости
):
    
    '''Bootstrap для непрерывной метрики'''
    
    boot_data = []
    for i in range(boot_it): # извлекаем подвыборки
#    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            len(data_column_1), 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            len(data_column_1), 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1)-statistic(samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
       
    return {"boot_data": boot_data, 
            "quants": quants, 
            "p_value": p_value}

def bootstrap_ratio(
        data: pd.DataFrame,
        x: str,
        y: str,
        x_f,
        y_f,
        split='EXP_GROUP',
        user_level_col='DRIVER_RK',
        boot_it=1000,
        conf_level=0.99):
    
    '''Bootstrap для метрики отношения'''
    
    data = data.sort_values(by=[split], ascending=[True])
    
    data_splitted = [x for _, x in data.groupby(split)]
    boot_data = []

    for i in range(boot_it):
        s0 = data_splitted[0][data_splitted[0][user_level_col].isin(
            data_splitted[0][user_level_col].sample(data_splitted[0][user_level_col].nunique(), replace=True))]
        s1 = data_splitted[1][data_splitted[1][user_level_col].isin(
            data_splitted[1][user_level_col].sample(data_splitted[1][user_level_col].nunique(), replace=True))]

        y0 = y_f(s0[y])
        y1 = y_f(s1[y])
        x0 = x_f(s0[x])
        x1 = x_f(s1[x])

        if y0 == 0 or y1 == 0:
            return None, False
        elif x0 == 0 or x1 == 0:
            return None, False

        s0_ratio = x0 / y0
        s1_ratio = x1 / y1
        boot_data.append(s1_ratio - s0_ratio)

    pd_boot_data = pd.DataFrame(boot_data)

    p_1 = norm.cdf(x=0, loc=np.mean(boot_data), scale=np.std(boot_data))
    p_2 = norm.cdf(x=0, loc=-np.mean(boot_data), scale=np.std(boot_data))
    pvalue = min(p_1, p_2) * 2
    mark = (pvalue < 1 - conf_level)

    return pvalue, mark

# Запрашиваем из БД все необходимые данные

In [4]:
cred = pd.read_json(r'/Users/skostuchik/crd_exa.json')
user = cred.iloc[0, 0]
password = cred.iloc[0, 1]
mysql_password = cred.iloc[0, 2]

C = pyexasol.connect(dsn='ex1..3.city-srv.ru:8563', user=user, password=password, fetch_dict=True)

In [31]:
alpha, power, n_bins = 0.01, 0.9, 2
localities_1 = '2'
localities_2 = '22394'
localities = localities_1 + ',' + localities_2
dates_1 = ['2021-09-29','2021-10-28']
dates_2 = ['2021-10-01','2021-10-28']
seed_1 = '_20210928'
seed_2 = '_20210930'

In [6]:
ar_query_1 = f'''
with s as (
    select
           row_number() over (partition by s.ID_DRIVER, s.ID_ORDER order by s.DATE_SUGGEST) rn
           , s.*
    from REPLICA.FAIRBOT_SUGGESTS_SUCCESS s
    where to_date(s.DATE_SUGGEST) between '{dates_1[0]}' and '{dates_1[1]}'
        --and s.SUGGEST_TYPE = 4
        ),
c as (
    select
           row_number() over (partition by c.DRIVER_ID, c.ORDER_ID order by c.EVENT_TIME) rn
           , c.*
    from REPLICA_CH.DRIVER_POINTS_REQUEST_CONVENIENCE c
    where to_date(c.EVENT_TIME) between '{dates_1[0]}' and '{dates_1[1]}')
select o.LOCALITY_RK, o.LOCALITY_NM, s.ID_DRIVER DRIVER_RK, to_date(s.DATE_SUGGEST) dt,
    case when mod(UDF.CRC32(CONCAT(s.ID_DRIVER, '{seed_1}')),100) < 50 then 'B' else 'A' end exp_group,
    s.ID fss_id,
    c.CONVENIENCE_TYPE, c.SPECIFICATION_NAME,
    case
        when o.EXP_DIST_KM between 0 and 3 then '0 - 3 km'
        when o.EXP_DIST_KM between 3 and 6 then '3 - 6 km'
        when o.EXP_DIST_KM between 6 and 10 then '6 - 10 km'
        when o.EXP_DIST_KM > 10 then '+10 km'
        else 'no data' end distance_category,
    case when s."ACTION" in (1,-2,3,4) then 1 else 0 end accept,
    case when s."ACTION" in (-1,-3) then 1 else 0 end "REJECT",
    case when s."ACTION" in (0,-4,-5) then 1 else 0 end fraud
from s
left join c on c.ORDER_ID = s.ID_ORDER and c.DRIVER_ID = s.ID_DRIVER and c.rn = s.rn
join emart."ORDER" o on o.ORDER_RK = s.ID_ORDER and o.LOCALITY_RK in ({localities_1})
'''

ar_query_2 = f'''
with s as (
    select
           row_number() over (partition by s.ID_DRIVER, s.ID_ORDER order by s.DATE_SUGGEST) rn
           , s.*
    from REPLICA.FAIRBOT_SUGGESTS_SUCCESS s
    where to_date(s.DATE_SUGGEST) between '{dates_2[0]}' and '{dates_2[1]}'
        --and s.SUGGEST_TYPE = 4
        ),
c as (
    select
           row_number() over (partition by c.DRIVER_ID, c.ORDER_ID order by c.EVENT_TIME) rn
           , c.*
    from REPLICA_CH.DRIVER_POINTS_REQUEST_CONVENIENCE c
    where to_date(c.EVENT_TIME) between '{dates_2[0]}' and '{dates_2[1]}')
select o.LOCALITY_RK, o.LOCALITY_NM, s.ID_DRIVER DRIVER_RK, to_date(s.DATE_SUGGEST) dt,
    case when mod(UDF.CRC32(CONCAT(s.ID_DRIVER, '{seed_2}')),100) < 50 then 'B' else 'A' end exp_group,
    s.ID fss_id,
    c.CONVENIENCE_TYPE, c.SPECIFICATION_NAME,
    case
        when o.EXP_DIST_KM between 0 and 3 then '0 - 3 km'
        when o.EXP_DIST_KM between 3 and 6 then '3 - 6 km'
        when o.EXP_DIST_KM between 6 and 10 then '6 - 10 km'
        when o.EXP_DIST_KM > 10 then '+10 km'
        else 'no data' end distance_category,
    case when s."ACTION" in (1,-2,3,4) then 1 else 0 end accept,
    case when s."ACTION" in (-1,-3) then 1 else 0 end "REJECT",
    case when s."ACTION" in (0,-4,-5) then 1 else 0 end fraud
from s
left join c on c.ORDER_ID = s.ID_ORDER and c.DRIVER_ID = s.ID_DRIVER and c.rn = s.rn
join emart."ORDER" o on o.ORDER_RK = s.ID_ORDER and o.LOCALITY_RK in ({localities_2})
'''

In [7]:
of2r_query_1 = f'''
with s as (
    select s.ID_DRIVER, count(s.ID) offers
    from REPLICA.FAIRBOT_SUGGESTS_SUCCESS s
    join replica.DRIVERS d on d.ID_LOCALITY IN ({localities_1}) and d.ID = s.ID_DRIVER
    where to_date(s.DATE_SUGGEST) between '{dates_1[0]}' and '{dates_1[1]}'
        --and s.SUGGEST_TYPE = 4
    group by 1),
o as (
    select o.DRIVER_RK, sum(o.STATUS_CD='CP') rides
    from EMART."ORDER" o
    WHERE to_date(o.LOCAL_ORDER_DTTM) between '{dates_1[0]}' and '{dates_1[1]}'
    group by 1)
select d.ID_LOCALITY LOCALITY_RK, l.SHORT_NAME LOCALITY_NM, s.ID_DRIVER DRIVER_RK,
       case when mod(UDF.CRC32(CONCAT(s.ID_DRIVER, '{seed_1}')), 100) < 50 then 'B' else 'A' end as exp_group,
       s.offers, ZEROIFNULL(o.rides) rides, cast(ZEROIFNULL(o.rides/s.offers) as float) OF2R
from s
left join o on s.ID_DRIVER = o.DRIVER_RK
join replica.DRIVERS d on d.ID = s.ID_DRIVER
join md.LOCALITY l on l.LOCALITY_RK = d.ID_LOCALITY
'''

of2r_query_2 = f'''
with s as (
    select s.ID_DRIVER, count(s.ID) offers
    from REPLICA.FAIRBOT_SUGGESTS_SUCCESS s
    join replica.DRIVERS d on d.ID_LOCALITY IN ({localities_2}) and d.ID = s.ID_DRIVER
    where to_date(s.DATE_SUGGEST) between '{dates_2[0]}' and '{dates_2[1]}'
        --and s.SUGGEST_TYPE = 4
    group by 1),
o as (
    select o.DRIVER_RK, sum(o.STATUS_CD='CP') rides
    from EMART."ORDER" o
    WHERE to_date(o.LOCAL_ORDER_DTTM) between '{dates_2[0]}' and '{dates_2[1]}'
    group by 1)
select d.ID_LOCALITY LOCALITY_RK, l.SHORT_NAME LOCALITY_NM, s.ID_DRIVER DRIVER_RK,
       case when mod(UDF.CRC32(CONCAT(s.ID_DRIVER, '{seed_2}')), 100) < 50 then 'B' else 'A' end as exp_group,
       s.offers, ZEROIFNULL(o.rides) rides, cast(ZEROIFNULL(o.rides/s.offers) as float) OF2R
from s
left join o on s.ID_DRIVER = o.DRIVER_RK
join replica.DRIVERS d on d.ID = s.ID_DRIVER
join md.LOCALITY l on l.LOCALITY_RK = d.ID_LOCALITY
'''

In [8]:
churn_query_1 = f'''
with lo as (
    select o.DRIVER_RK, max(o.LOCAL_ORDER_DTTM) last_order
    from EMART."ORDER" o
    where o.LOCALITY_RK IN ({localities_1})
        and o.STATUS_CD = 'CP'
    group by o.DRIVER_RK),
p as (
    select df.driver_rk, df.is_park, df.trips,
           row_number() over (partition by df.DRIVER_RK order by df.trips desc) rn
    from (
    SELECT o.DRIVER_RK,
           case when lower(o.COMPANY_BRAND_NM) like '%парк%' then 'Park' else 'Not_park' end is_park,
           count(o.ORDER_RK) trips
    FROM EMART.order o
    where STATUS_CD='CP'
            and to_date(o.LOCAL_ORDER_DTTM) >= '{dates_1[0]}'
            and o.DRIVER_RK is not null
    group by 1, 2) df),
rfm as (
    SELECT ds.DRIVER_ID, ds.SEGMENT_ID,
           case
                when ds.SEGMENT_ID = 0 then 'BOTTOM'
                when ds.SEGMENT_ID = 1 then 'LOW'
                when ds.SEGMENT_ID = 2 then 'MIDDLE'
                when ds.SEGMENT_ID = 3 then 'HIGH'
                when ds.SEGMENT_ID = 4 then 'TOP'
                end segment
    FROM REPLICA.DRIVER_SEGMENTATION_RFM ds
    where ds."DATE" = '{dates_1[0]}')
select d.ID_LOCALITY LOCALITY_RK, l.SHORT_NAME LOCALITY_NM, lo.DRIVER_RK, p.is_park, rfm.segment,
       case when mod(UDF.CRC32(CONCAT(lo.DRIVER_RK, '{seed_1}')), 100) < 50 then 'B'
           else 'A' end as exp_group,
       case when lo.last_order between '{dates_1[0]}' and '{str((pd.to_datetime(dates_1[0]) + pd.DateOffset(days=6)).date())}' then 1 else 0 end churn_7day,
       case when lo.last_order between '{dates_1[0]}' and '{str((pd.to_datetime(dates_1[0]) + pd.DateOffset(days=13)).date())}' then 1 else 0 end churn_14day,
       case when lo.last_order between '{dates_1[0]}' and '{str((pd.to_datetime(dates_1[0]) + pd.DateOffset(days=20)).date())}' then 1 else 0 end churn_21day
from lo
join replica.DRIVERS d on d.ID = lo.DRIVER_RK
left join p on p.DRIVER_RK = lo.DRIVER_RK and p.rn = 1
left join rfm on rfm.DRIVER_ID = lo.DRIVER_RK
left join md.LOCALITY l on l.LOCALITY_RK = d.ID_LOCALITY
where lo.last_order >= '{dates_1[0]}'
'''

churn_query_2 = f'''
with lo as (
    select o.DRIVER_RK, max(o.LOCAL_ORDER_DTTM) last_order
    from EMART."ORDER" o
    where o.LOCALITY_RK IN ({localities_2})
        and o.STATUS_CD = 'CP'
    group by o.DRIVER_RK),
p as (
    select df.driver_rk, df.is_park, df.trips,
           row_number() over (partition by df.DRIVER_RK order by df.trips desc) rn
    from (
    SELECT o.DRIVER_RK,
           case when lower(o.COMPANY_BRAND_NM) like '%парк%' then 'Park' else 'Not_park' end is_park,
           count(o.ORDER_RK) trips
    FROM EMART.order o
    where STATUS_CD='CP'
            and to_date(o.LOCAL_ORDER_DTTM) >= '{dates_2[0]}'
            and o.DRIVER_RK is not null
    group by 1, 2) df),
rfm as (
    SELECT ds.DRIVER_ID, ds.SEGMENT_ID,
           case
                when ds.SEGMENT_ID = 0 then 'BOTTOM'
                when ds.SEGMENT_ID = 1 then 'LOW'
                when ds.SEGMENT_ID = 2 then 'MIDDLE'
                when ds.SEGMENT_ID = 3 then 'HIGH'
                when ds.SEGMENT_ID = 4 then 'TOP'
                end segment
    FROM REPLICA.DRIVER_SEGMENTATION_RFM ds
    where ds."DATE" = '{dates_2[0]}')
select d.ID_LOCALITY LOCALITY_RK, l.SHORT_NAME LOCALITY_NM, lo.DRIVER_RK, p.is_park, rfm.segment,
       case when mod(UDF.CRC32(CONCAT(lo.DRIVER_RK, '{seed_2}')), 100) < 50 then 'B'
           else 'A' end as exp_group,
       case when lo.last_order between '{dates_2[0]}' and '{str((pd.to_datetime(dates_2[0]) + pd.DateOffset(days=6)).date())}' then 1 else 0 end churn_7day,
       case when lo.last_order between '{dates_2[0]}' and '{str((pd.to_datetime(dates_2[0]) + pd.DateOffset(days=13)).date())}' then 1 else 0 end churn_14day,
       case when lo.last_order between '{dates_2[0]}' and '{str((pd.to_datetime(dates_2[0]) + pd.DateOffset(days=20)).date())}' then 1 else 0 end churn_21day
from lo
join replica.DRIVERS d on d.ID = lo.DRIVER_RK
left join p on p.DRIVER_RK = lo.DRIVER_RK and p.rn = 1
left join rfm on rfm.DRIVER_ID = lo.DRIVER_RK
left join md.LOCALITY l on l.LOCALITY_RK = d.ID_LOCALITY
where lo.last_order >= '{dates_2[0]}'
'''

In [9]:
mph_query_1 = f'''
with sh as (
    select DRIVER_RK, cast(ZEROIFNULL(SUM(DRIVER_STAY_DUR_SEC)/3600) as float) supply_hours
    from REPLICA_MART.DRIVER_MOVING_AGG_DAILY
    where STATE_TCD in (1,2,3)
      and DRIVER_ROBOT_CD != 0
        and BUSINESS_DT between '{dates_1[0]}' and '{dates_1[1]}'
    group by 1),
o as (
    select o.DRIVER_RK,
           case when mod(UDF.CRC32(CONCAT(o.DRIVER_RK, '{seed_1}')), 100) < 50 then 'B'
                   else 'A' end as exp_group,
           cast(ZEROIFNULL(SUM(ic.driver_bill_amt +ic.di_mfg + ic.di_welcome_dxgy + ic.di_power_dxgy
               + ic.di_other_dxgy + ic.di_main_dxgy+ ic.di_guaranteed_amt_per_hour)) as float) money,
        cast(ZEROIFNULL(sum(case when o.STATUS_CD='CP' then 1 else 0 end)) as float) trips
    from EMART."ORDER" o
    left join replica_mart.incentive_comission ic on o.order_rk = ic.ORDER_ID
    WHERE to_date(o.LOCAL_ORDER_DTTM) between '{dates_1[0]}' and '{dates_1[1]}'
        and o.LOCALITY_RK IN ({localities_1})
    group by 1)
select d.ID_LOCALITY LOCALITY_RK, l.short_name LOCALITY_NM, o.DRIVER_RK, o.exp_group, o.money, sh.supply_hours,
    o.money/sh.supply_hours MpH_Gross, trips
from o
join REPLICA.DRIVERS d on d.ID = o.DRIVER_RK
left join md.LOCALITY l on l.LOCALITY_RK = d.ID_LOCALITY
left join sh on sh.DRIVER_RK = o.DRIVER_RK
'''

mph_query_2 = f'''
with sh as (
    select DRIVER_RK, cast(ZEROIFNULL(SUM(DRIVER_STAY_DUR_SEC)/3600) as float) supply_hours
    from REPLICA_MART.DRIVER_MOVING_AGG_DAILY
    where STATE_TCD in (1,2,3)
      and DRIVER_ROBOT_CD != 0
        and BUSINESS_DT between '{dates_2[0]}' and '{dates_2[1]}'
    group by 1),
o as (
    select o.DRIVER_RK,
           case when mod(UDF.CRC32(CONCAT(o.DRIVER_RK, '{seed_2}')), 100) < 50 then 'B'
                   else 'A' end as exp_group,
           cast(ZEROIFNULL(SUM(ic.driver_bill_amt +ic.di_mfg + ic.di_welcome_dxgy + ic.di_power_dxgy
               + ic.di_other_dxgy + ic.di_main_dxgy+ ic.di_guaranteed_amt_per_hour)) as float) money,
        cast(ZEROIFNULL(sum(case when o.STATUS_CD='CP' then 1 else 0 end)) as float) trips
    from EMART."ORDER" o
    left join replica_mart.incentive_comission ic on o.order_rk = ic.ORDER_ID
    WHERE to_date(o.LOCAL_ORDER_DTTM) between '{dates_2[0]}' and '{dates_2[1]}'
        and o.LOCALITY_RK IN ({localities_2})
    group by 1)
select d.ID_LOCALITY LOCALITY_RK, l.short_name LOCALITY_NM, o.DRIVER_RK, o.exp_group, o.money, sh.supply_hours,
    o.money/sh.supply_hours MpH_Gross, trips
from o
join REPLICA.DRIVERS d on d.ID = o.DRIVER_RK
left join md.LOCALITY l on l.LOCALITY_RK = d.ID_LOCALITY
left join sh on sh.DRIVER_RK = o.DRIVER_RK
'''

In [10]:
copt_query_1 = f'''
select o.LOCALITY_RK, o.LOCALITY_NM, o.DRIVER_RK,
       case when mod(UDF.CRC32(CONCAT(o.DRIVER_RK, '{seed_1}')), 100) < 50 then 'B' else 'A' end as exp_group,
       cast(count(o.ORDER_RK) as float) rides,
       cast(sum(ZEROIFNULL(ic.COMMISSION_TOTAL_AMT - ic.DI_TOTAL_AMT - ic.CI_TOTAL_AMT)) as float) contribution,
       cast(sum(ZEROIFNULL(ic.DI_TOTAL_AMT)) as float) DI,
       cast(sum(ZEROIFNULL(ic.COMMISSION_TOTAL_AMT)) as float) COMMISSION,
       cast(ZEROIFNULL(SUM(ic.COMMISSION_TOTAL_AMT - ic.DI_TOTAL_AMT - ic.CI_TOTAL_AMT)/
            count(o.ORDER_RK)) as float) COPT,
       cast(ZEROIFNULL(SUM(ic.DI_TOTAL_AMT)/count(o.ORDER_RK)) as float) DIPT,
       cast(ZEROIFNULL(SUM(ic.COMMISSION_TOTAL_AMT)/count(o.ORDER_RK)) as float) COMPT
from EMART."ORDER" o
LEFT JOIN REPLICA_MART.INCENTIVE_COMISSION ic on ic.ORDER_ID = o.ORDER_RK and ic.DRIVER_ID = o.DRIVER_RK
WHERE to_date(o.LOCAL_ORDER_DTTM) between '{dates_1[0]}' and '{dates_1[1]}'
    AND o.STATUS_CD = 'CP'
    AND o.LOCALITY_RK IN ({localities_1})
    AND o.DRIVER_RK is not null
group by 3,1,2
'''

copt_query_2 = f'''
select o.LOCALITY_RK, o.LOCALITY_NM, o.DRIVER_RK,
       case when mod(UDF.CRC32(CONCAT(o.DRIVER_RK, '{seed_2}')), 100) < 50 then 'B' else 'A' end as exp_group,
       cast(count(o.ORDER_RK) as float) rides,
       cast(sum(ZEROIFNULL(ic.COMMISSION_TOTAL_AMT - ic.DI_TOTAL_AMT - ic.CI_TOTAL_AMT)) as float) contribution,
       cast(sum(ZEROIFNULL(ic.DI_TOTAL_AMT)) as float) DI,
       cast(sum(ZEROIFNULL(ic.COMMISSION_TOTAL_AMT)) as float) COMMISSION,
       cast(ZEROIFNULL(SUM(ic.COMMISSION_TOTAL_AMT - ic.DI_TOTAL_AMT - ic.CI_TOTAL_AMT)/
            count(o.ORDER_RK)) as float) COPT,
       cast(ZEROIFNULL(SUM(ic.DI_TOTAL_AMT)/count(o.ORDER_RK)) as float) DIPT,
       cast(ZEROIFNULL(SUM(ic.COMMISSION_TOTAL_AMT)/count(o.ORDER_RK)) as float) COMPT
from EMART."ORDER" o
LEFT JOIN REPLICA_MART.INCENTIVE_COMISSION ic on ic.ORDER_ID = o.ORDER_RK and ic.DRIVER_ID = o.DRIVER_RK
WHERE to_date(o.LOCAL_ORDER_DTTM) between '{dates_2[0]}' and '{dates_2[1]}'
    AND o.STATUS_CD = 'CP'
    AND o.LOCALITY_RK IN ({localities_2})
    AND o.DRIVER_RK is not null
group by 3,1,2
'''

In [11]:
e_ar_1, e_of2r_1, e_churn_1, e_mph_1, e_copt_1 = C.execute(ar_query_1), C.execute(of2r_query_1), C.execute(churn_query_1), C.execute(mph_query_1), C.execute(copt_query_1)

In [12]:
ar_suggest_data_1, of2r_data_1, churn_data_1, mph_data_1, copt_data_1 = pd.DataFrame(e_ar_1.fetchall()), pd.DataFrame(e_of2r_1.fetchall()), pd.DataFrame(e_churn_1.fetchall()), pd.DataFrame(e_mph_1.fetchall()), pd.DataFrame(e_copt_1.fetchall())

In [13]:
e_ar_2, of2r_2, e_churn_2, e_mph_2, e_copt_2 = C.execute(ar_query_2), C.execute(of2r_query_2), C.execute(churn_query_2), C.execute(mph_query_2), C.execute(copt_query_2)

In [14]:
ar_suggest_data_2, of2r_data_2, churn_data_2, mph_data_2, copt_data_2 = pd.DataFrame(e_ar_2.fetchall()), pd.DataFrame(of2r_2.fetchall()), pd.DataFrame(e_churn_2.fetchall()), pd.DataFrame(e_mph_2.fetchall()), pd.DataFrame(e_copt_2.fetchall())

In [15]:
ar_suggest_data, of2r_data, churn_data, mph_data, copt_data = ar_suggest_data_1.append(ar_suggest_data_2), of2r_data_1.append(of2r_data_2), churn_data_1.append(churn_data_2), mph_data_1.append(mph_data_2), copt_data_1.append(copt_data_2)

In [16]:
print(ar_suggest_data.isnull().sum(),'\n')
print(of2r_data.isnull().sum(),'\n')
print(churn_data.isnull().sum(),'\n')
print(mph_data.isnull().sum(),'\n')
print(copt_data.isnull().sum(),'\n')

LOCALITY_RK               0
LOCALITY_NM               0
DRIVER_RK                 0
DT                        0
EXP_GROUP                 0
FSS_ID                    0
CONVENIENCE_TYPE      30882
SPECIFICATION_NAME    30882
DISTANCE_CATEGORY         0
ACCEPT                    0
REJECT                    0
FRAUD                     0
dtype: int64 

LOCALITY_RK    0
LOCALITY_NM    0
DRIVER_RK      0
EXP_GROUP      0
OFFERS         0
RIDES          0
OF2R           0
dtype: int64 

LOCALITY_RK        0
LOCALITY_NM        0
DRIVER_RK          0
IS_PARK            0
SEGMENT        33363
EXP_GROUP          0
CHURN_7DAY         0
CHURN_14DAY        0
CHURN_21DAY        0
dtype: int64 

LOCALITY_RK     0
LOCALITY_NM     0
DRIVER_RK       0
EXP_GROUP       0
MONEY           0
SUPPLY_HOURS    9
MPH_GROSS       9
TRIPS           0
dtype: int64 

LOCALITY_RK     0
LOCALITY_NM     0
DRIVER_RK       0
EXP_GROUP       0
RIDES           0
CONTRIBUTION    0
DI              0
COMMISSION      0
COPT    

In [17]:
mph_data.dropna(inplace=True)

In [27]:
config = {
    'metrics': {
        'AR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['ACCEPT'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'RR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['REJECT'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'FR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['FRAUD'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_7DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total',
                'Park':'IS_PARK',
                'HIGH':'SEGMENT','TOP':'SEGMENT'
            },
            'options':{
                'x':['CHURN_7DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_14DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total',
                'Park':'IS_PARK',
                'HIGH':'SEGMENT','TOP':'SEGMENT'
            },
            'options':{
                'x':['CHURN_14DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_21DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total',
                'Park':'IS_PARK',
                'HIGH':'SEGMENT','TOP':'SEGMENT'
            },
            'options':{
                'x':['CHURN_21DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'MPH_GROSS':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['MONEY', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'SHpD':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['SUPPLY_HOURS', np.sum],
                'y':['DRIVER_RK', pd.Series.nunique],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'TPD':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['TRIPS', np.sum],
                'y':['DRIVER_RK', pd.Series.nunique],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'OF2R':{
            'df':of2r_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['RIDES', np.sum],
                'y':['OFFERS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'COPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CONTRIBUTION', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'DIPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['DI', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'COMPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['COMMISSION', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        }
    }
}

In [28]:
def stat_res_calculation(metric: str, m_type: str, df: pd.DataFrame, cfg=config):
    '''Возвращает массив со всеми расчитанными метриками'''
    if m_type == 'binomial':
        a_s = len(df[(df['EXP_GROUP'] == 'A')&(df[config['metrics'][metric]['options']['x'][0]] > 0)])
        a_f = len(df[(df['EXP_GROUP'] == 'A')&(df[config['metrics'][metric]['options']['x'][0]] == 0)])
        b_s = len(df[(df['EXP_GROUP'] == 'B')&(df[config['metrics'][metric]['options']['x'][0]] > 0)])
        b_f = len(df[(df['EXP_GROUP'] == 'B')&(df[config['metrics'][metric]['options']['x'][0]] == 0)])
        
        act_nobs_a, act_nobs_b = a_s+a_f, b_s+b_f
        successes, fails = [a_s, b_s], [a_f, b_f]
        probs0, probs1 = np.array([a_s, a_f]), np.array([b_s, b_f])
        effect_size = chisquare_effectsize(probs0, probs1, correction=None, cohen=True, axis=0)
        
        p_val = chi2_contingency(np.array([fails, successes]))[1]
        mde = int(GofChisquarePower().solve_power(effect_size=effect_size, alpha=alpha,
                                                  power=power, n_bins=n_bins))
        mde_marker_value = 1 if (act_nobs_a>=mde)&(act_nobs_b>=mde) else np.nan
        
        a = a_s/act_nobs_a
        b = b_s/act_nobs_b
        f = config['metrics'][metric]['options']['n'][1]
        a_drivers = df[df['EXP_GROUP'] == 'A']['DRIVER_RK'].nunique()
        b_drivers = df[df['EXP_GROUP'] == 'B']['DRIVER_RK'].nunique()
        lift = b-a

        
    elif m_type == 'ratio':
        x = config['metrics'][metric]['options']['x'][0]
        x_f = config['metrics'][metric]['options']['x'][1]
        y = config['metrics'][metric]['options']['y'][0]
        y_f = config['metrics'][metric]['options']['y'][1]
        
        f = config['metrics'][metric]['options']['n'][1]
        act_nobs_a = f(df[df['EXP_GROUP'] == 'A'][config['metrics'][metric]['options']['n'][0]])
        act_nobs_b = f(df[df['EXP_GROUP'] == 'B'][config['metrics'][metric]['options']['n'][0]])
        
        mde_ttl = df.groupby('EXP_GROUP').mean().reset_index()
        mean = df[df['EXP_GROUP']=='A'][x].mean()
        sd = df[df['EXP_GROUP']=='A'][x].std()
        lift = mde_ttl[x][1] / mde_ttl[x][0]-1
        effect_size = mean / sd * lift
        
        p_val = bootstrap_ratio(data=df, x=x, y=y, x_f=x_f, y_f=y_f)[0]
        mde = int(tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, nobs1=None, ratio=1))
        mde_marker_value = 1 if (act_nobs_a>=mde)&(act_nobs_b>=mde) else np.nan
        
        a = x_f(df[df['EXP_GROUP']=='A'][x]) / y_f(df[df['EXP_GROUP']=='A'][y])
        b = x_f(df[df['EXP_GROUP']=='B'][x]) / y_f(df[df['EXP_GROUP']=='B'][y])
        a_drivers, b_drivers = act_nobs_a, act_nobs_b
        lift = (b-a)/abs(a)

    else:
        return('Unknown metric type')
    
    nobs_diff_value = (act_nobs_b-act_nobs_a)/abs(act_nobs_a)
    dn_diff_value = (b_drivers-a_drivers)/abs(a_drivers)
    
    return({'p_val':p_val, 'nobs_needed':mde, 'mde_ok':mde_marker_value,
            'split_a':a, 'split_b':b, 'lift':lift, 'split_a_n_obs':act_nobs_a, 'split_b_n_obs':act_nobs_b,
            'n_obs_difference':nobs_diff_value, 'split_a_drivers':a_drivers, 'split_b_drivers':b_drivers,
            'unique_drivers_difference':dn_diff_value})

In [29]:
result_dict = {}

for l in tqdm(localities.split(',')):
    for metric,v in config.get('metrics').items():
        df = v.get('df')
        metric_data = df[df['LOCALITY_RK']==int(l)]
        for value, fltr in v.get('add_breakdown').items():
            if fltr != 'Total':
                add_breakdown_data = metric_data[metric_data[fltr] == value]
            else:
                add_breakdown_data = metric_data
            #print(len(add_breakdown_data))
            locality = add_breakdown_data['LOCALITY_NM'].iloc[0]
            
            metric_name = metric+'_'+value if len(v.get('add_breakdown')) > 1 else metric
            

                   
            #print(locality, metric, fltr, value, metric_name)
            #print(locality, metric_name)
            
            try:
                src = stat_res_calculation(metric=metric, m_type=v.get('type'), df=add_breakdown_data)
                result_dict.update({(locality,metric_name):src})
            except:
                continue

100%|██████████| 2/2 [02:06<00:00, 63.47s/it] 


In [32]:
experiment_summary = pd.DataFrame(result_dict).T

# Делаем поправку БХ на множественные сравнения

In [41]:
p_res = pd.DataFrame(experiment_summary['p_val'])

In [43]:
p_adj_list = {}

for locality in p_res.index.get_level_values(0).unique():

    loc_index = p_res[p_res.index.get_level_values(0)==locality].index
    loc_p = p_res[p_res.index.get_level_values(0)==locality]['p_val']
    loc_p_adj = multipletests(loc_p, alpha=0.01, method='fdr_bh')[1]
    loc_dict = dict(list(zip(loc_index,zip(loc_p,loc_p_adj))))
    
    p_adj_list.update(loc_dict)

In [44]:
p_adj = pd.DataFrame(p_adj_list, index=['p_val','p_adj']).T

# Итоги теста:

In [49]:
experiment_summary = experiment_summary.join(p_adj['p_adj'])
experiment_summary = experiment_summary.reset_index().rename(columns={"level_0": "locality", "level_1": "metric"})
experiment_summary = experiment_summary[[
    'locality', 'metric', 'split_a', 'split_b', 'lift', 'split_a_n_obs', 'split_b_n_obs', 'n_obs_difference',
    'split_a_drivers', 'split_b_drivers', 'unique_drivers_difference', 'p_val', 'p_adj', 'nobs_needed', 'mde_ok']]

In [56]:
experiment_summary

,locality,metric,split_a,split_b,lift,split_a_n_obs,split_b_n_obs,n_obs_difference,split_a_drivers,split_b_drivers,unique_drivers_difference,p_val,p_adj,nobs_needed,mde_ok
0,Москва,AR_Total,0.529600,0.564405,0.034805,4474272.0,4128224.0,-0.077342,25647.0,25627.0,-0.000780,0.000000e+00,0.000000e+00,3059.0,1.0
1,Москва,AR_ChainOffer,0.634141,0.822845,0.188705,625134.0,520269.0,-0.167748,17208.0,17195.0,-0.000755,0.000000e+00,0.000000e+00,96.0,1.0
2,Москва,AR_0 - 3 km,0.754223,0.777521,0.023298,1079996.0,1006971.0,-0.067616,21474.0,21422.0,-0.002422,0.000000e+00,0.000000e+00,5081.0,1.0
3,Москва,AR_3 - 6 km,0.641917,0.671079,0.029162,887238.0,834020.0,-0.059982,21330.0,21402.0,0.003376,0.000000e+00,0.000000e+00,4021.0,1.0
4,Москва,AR_6 - 10 km,0.512929,0.548376,0.035447,720406.0,671038.0,-0.068528,20868.0,20868.0,0.000000,0.000000e+00,0.000000e+00,2958.0,1.0
5,Москва,AR_+10 km,0.344764,0.383231,0.038467,1786632.0,1616195.0,-0.095396,22492.0,22514.0,0.000978,0.000000e+00,0.000000e+00,2271.0,1.0
6,Москва,RR_Total,0.417587,0.382522,-0.035065,4474272.0,4128224.0,-0.077342,25647.0,25627.0,-0.000780,0.000000e+00,0.000000e+00,2943.0,1.0
7,Москва,RR_ChainOffer,0.347794,0.144291,-0.203503,625134.0,520269.0,-0.167748,17208.0,17195.0,-0.000755,0.000000e+00,0.000000e+00,81.0,1.0
8,Москва,RR_0 - 3 km,0.209831,0.186448,-0.023383,1079996.0,1006971.0,-0.067616,21474.0,21422.0,-0.002422,0.000000e+00,0.000000e+00,4512.0,1.0
9,Москва,RR_3 - 6 km,0.315670,0.287029,-0.028640,887238.0,834020.0,-0.059982,21330.0,21402.0,0.003376,0.000000e+00,0.000000e+00,3918.0,1.0


In [57]:
#словарь для порядка сортировки
m_order = {'AR_Total':0, 'RR_Total':1, 'FR_Total':2, 'AR_ChainOffer':3, 'RR_ChainOffer':4, 'FR_ChainOffer':5,
           'AR_0 - 3 km':6, 'AR_3 - 6 km':7, 'AR_6 - 10 km':8, 'AR_+10 km':9, 'RR_0 - 3 km':10, 
           'RR_3 - 6 km':11, 'RR_6 - 10 km':12, 'RR_+10 km':13, 'FR_0 - 3 km':14, 'FR_3 - 6 km':15,
           'FR_6 - 10 km':16, 'FR_+10 km':17, 'OF2R':18, 'CHURN_7DAY_Total':19, 'CHURN_14DAY_Total':20,
           'CHURN_21DAY_Total':20.1,
           'CHURN_7DAY_Park':21, 'CHURN_14DAY_Park':22,
           'CHURN_21DAY_Park':22.1,
           'CHURN_7DAY_HIGH':23, 'CHURN_14DAY_HIGH':24,
           'CHURN_21DAY_HIGH':24.1,
           'CHURN_7DAY_TOP':25, 'CHURN_14DAY_TOP':26,
           'CHURN_21DAY_TOP':26.1,
           'MPH_GROSS':27, 'SHPD':28, 'TPD':29,'COPT':30,
           'DIPT':31, 'COMPT':32}

In [58]:
#добавляем города в порядок сортировки
locs_dict = dict(list(enumerate(experiment_summary.locality.unique())))
locs_dict = dict((v,k) for k,v in locs_dict.items())
m_order.update(locs_dict)

In [59]:
experiment_summary = experiment_summary.sort_values(by=['locality','metric'], key=lambda x: x.map(m_order))
prokras = experiment_summary[(experiment_summary['p_adj']<=0.01)&(experiment_summary['mde_ok']==1)]

In [60]:
experiment_summary.drop(['unique_drivers_difference'],axis=1)

,locality,metric,split_a,split_b,lift,split_a_n_obs,split_b_n_obs,n_obs_difference,split_a_drivers,split_b_drivers,p_val,p_adj,nobs_needed,mde_ok
0,Москва,AR_Total,0.529600,0.564405,0.034805,4474272.0,4128224.0,-0.077342,25647.0,25627.0,0.000000e+00,0.000000e+00,3059.0,1.0
6,Москва,RR_Total,0.417587,0.382522,-0.035065,4474272.0,4128224.0,-0.077342,25647.0,25627.0,0.000000e+00,0.000000e+00,2943.0,1.0
12,Москва,FR_Total,0.052786,0.053047,0.000261,4474272.0,4128224.0,-0.077342,25647.0,25627.0,8.755575e-02,1.463291e-01,10901900.0,NaN
1,Москва,AR_ChainOffer,0.634141,0.822845,0.188705,625134.0,520269.0,-0.167748,17208.0,17195.0,0.000000e+00,0.000000e+00,96.0,1.0
7,Москва,RR_ChainOffer,0.347794,0.144291,-0.203503,625134.0,520269.0,-0.167748,17208.0,17195.0,0.000000e+00,0.000000e+00,81.0,1.0
13,Москва,FR_ChainOffer,0.018065,0.032862,0.014797,625134.0,520269.0,-0.167748,17208.0,17195.0,0.000000e+00,0.000000e+00,1205.0,1.0
2,Москва,AR_0 - 3 km,0.754223,0.777521,0.023298,1079996.0,1006971.0,-0.067616,21474.0,21422.0,0.000000e+00,0.000000e+00,5081.0,1.0
3,Москва,AR_3 - 6 km,0.641917,0.671079,0.029162,887238.0,834020.0,-0.059982,21330.0,21402.0,0.000000e+00,0.000000e+00,4021.0,1.0
4,Москва,AR_6 - 10 km,0.512929,0.548376,0.035447,720406.0,671038.0,-0.068528,20868.0,20868.0,0.000000e+00,0.000000e+00,2958.0,1.0
5,Москва,AR_+10 km,0.344764,0.383231,0.038467,1786632.0,1616195.0,-0.095396,22492.0,22514.0,0.000000e+00,0.000000e+00,2271.0,1.0


In [61]:
prokras.drop(['unique_drivers_difference'],axis=1)

,locality,metric,split_a,split_b,lift,split_a_n_obs,split_b_n_obs,n_obs_difference,split_a_drivers,split_b_drivers,p_val,p_adj,nobs_needed,mde_ok
0,Москва,AR_Total,0.529600,0.564405,0.034805,4474272.0,4128224.0,-0.077342,25647.0,25627.0,0.000000e+00,0.000000e+00,3059.0,1.0
6,Москва,RR_Total,0.417587,0.382522,-0.035065,4474272.0,4128224.0,-0.077342,25647.0,25627.0,0.000000e+00,0.000000e+00,2943.0,1.0
1,Москва,AR_ChainOffer,0.634141,0.822845,0.188705,625134.0,520269.0,-0.167748,17208.0,17195.0,0.000000e+00,0.000000e+00,96.0,1.0
7,Москва,RR_ChainOffer,0.347794,0.144291,-0.203503,625134.0,520269.0,-0.167748,17208.0,17195.0,0.000000e+00,0.000000e+00,81.0,1.0
13,Москва,FR_ChainOffer,0.018065,0.032862,0.014797,625134.0,520269.0,-0.167748,17208.0,17195.0,0.000000e+00,0.000000e+00,1205.0,1.0
2,Москва,AR_0 - 3 km,0.754223,0.777521,0.023298,1079996.0,1006971.0,-0.067616,21474.0,21422.0,0.000000e+00,0.000000e+00,5081.0,1.0
3,Москва,AR_3 - 6 km,0.641917,0.671079,0.029162,887238.0,834020.0,-0.059982,21330.0,21402.0,0.000000e+00,0.000000e+00,4021.0,1.0
4,Москва,AR_6 - 10 km,0.512929,0.548376,0.035447,720406.0,671038.0,-0.068528,20868.0,20868.0,0.000000e+00,0.000000e+00,2958.0,1.0
5,Москва,AR_+10 km,0.344764,0.383231,0.038467,1786632.0,1616195.0,-0.095396,22492.0,22514.0,0.000000e+00,0.000000e+00,2271.0,1.0
8,Москва,RR_0 - 3 km,0.209831,0.186448,-0.023383,1079996.0,1006971.0,-0.067616,21474.0,21422.0,0.000000e+00,0.000000e+00,4512.0,1.0


# Как эксперимент повлияет на экономику в городе

In [62]:
es = experiment_summary[experiment_summary['metric'].isin(['CHURN_21DAY_Total','TPD','COPT','COMPT'])][[
    'locality','metric','split_a','split_b','split_a_drivers']]

In [63]:
es = es[['locality','metric','split_a','split_b']].melt(
    id_vars=['locality','metric'], var_name='split', value_name='values').pivot_table(
    values='values', index=['locality', 'split'], columns=['metric']).reset_index(level='split').join(
    es[['locality','split_a_drivers']].groupby('locality').max()).reset_index()

es.columns = ['locality', 'split','CHURN', 'COMPT','COPT','TPD','drivers']

es['TRIPS'] = es['drivers'] * (1 - es['CHURN']) * es['TPD']
es['contribution'] = es['TRIPS'] * es['COPT']
es['comission'] = es['TRIPS'] * es['COMPT']

In [64]:
es = es[['locality', 'split', 'drivers', 'CHURN', 'TPD', 'TRIPS', 'COPT', 'contribution', 'COMPT', 'comission']]

In [65]:
es

,locality,split,drivers,CHURN,TPD,TRIPS,COPT,contribution,COMPT,comission
0,Москва,split_a,26402.0,0.251269,84.368762,1.667802e+06,17.604827,2.936136e+07,91.041386,1.518390e+08
1,Москва,split_b,26402.0,0.251208,82.790857,1.636743e+06,19.566039,3.202457e+07,90.097114,1.474658e+08
2,Тольятти,split_a,1131.0,0.229001,119.200579,1.039429e+05,9.081617,9.439697e+05,26.265920,2.730156e+06
3,Тольятти,split_b,1131.0,0.281389,91.290891,7.419657e+04,14.015485,1.039901e+06,26.539747,1.969158e+06


In [ ]:
#экспорт в эксель
experiment_summary.to_excel(r'res_ttl.xlsx', sheet_name='experiment_summary', index = False)
prokras.to_excel(r'res_prokras.xlsx', sheet_name='prokras', index = False)
mde.reset_index().to_excel(r'res_mde.xlsx', sheet_name='mde', index = False)
es.to_excel(r'economy.xlsx', sheet_name='economy', index = False)